# Model Metrics Overview: GR4J, HBV, RFR with Different Meteo Sources

Overview of performance metrics for different hydrological models trained with different meteorological data sources.

In [1]:
import json
from pathlib import Path

import numpy as np
import pandas as pd

# Base paths for optimization results
BASE_PATHS = {
    "GR4J": Path("/home/dmbrmv/Development/Dissertation/data/optimization/gr4j_simple"),
    "HBV": Path("/home/dmbrmv/Development/Dissertation/data/optimization/hbv_simple"),
    "RFR": Path("/home/dmbrmv/Development/Dissertation/data/optimization/rfr_simple"),
}

## Load Model Results and Metrics

Extract performance metrics from all model directories across different meteorological sources.

In [2]:
def load_metrics_from_directory(base_path: Path) -> dict:
    """Load all metrics from model optimization directory.

    Args:
        base_path: Path to model optimization directory.

    Returns:
        Dictionary with structure {gauge_id: {meteo_source: metrics_dict}}.
    """
    results = {}

    for gauge_dir in base_path.iterdir():
        if not gauge_dir.is_dir():
            continue

        gauge_id = gauge_dir.name
        results[gauge_id] = {}

        # Find all metrics files
        for metrics_file in gauge_dir.glob("*_metrics.json"):
            # Extract meteo source from filename (e.g., "10042_e5_metrics.json")
            parts = metrics_file.stem.split("_")
            meteo_source = "_".join(parts[1:-1])  # Get middle part(s)

            try:
                with open(metrics_file) as f:
                    metrics = json.load(f)
                    results[gauge_id][meteo_source] = metrics
            except (json.JSONDecodeError, FileNotFoundError):
                continue

    return results


# Load data for all models
model_data = {}
for model_name, base_path in BASE_PATHS.items():
    print(f"Loading {model_name} results from {base_path}...")
    model_data[model_name] = load_metrics_from_directory(base_path)
    print(f"  Found {len(model_data[model_name])} gauges")

print("\nSample meteo sources found:")
for model_name, data in model_data.items():
    if data:
        first_gauge = next(iter(data.values()))
        print(f"  {model_name}: {list(first_gauge.keys())}")

Loading GR4J results from /home/dmbrmv/Development/Dissertation/data/optimization/gr4j_simple...
  Found 637 gauges
Loading HBV results from /home/dmbrmv/Development/Dissertation/data/optimization/hbv_simple...
  Found 37 gauges
Loading RFR results from /home/dmbrmv/Development/Dissertation/data/optimization/rfr_simple...
  Found 0 gauges

Sample meteo sources found:
  GR4J: ['mswep', 'gpcp', 'e5l', 'e5']
  HBV: []


## Parse Meteorological Sources

Identify and extract meteorological source information from the model results.

In [3]:
# Identify all unique meteo sources across all models
all_meteo_sources = set()
for model_data_dict in model_data.values():
    for gauge_metrics in model_data_dict.values():
        all_meteo_sources.update(gauge_metrics.keys())

all_meteo_sources = sorted(list(all_meteo_sources))
print(f"All meteorological sources found: {all_meteo_sources}")
print(f"Total sources: {len(all_meteo_sources)}")

All meteorological sources found: ['e5', 'e5l', 'gpcp', 'mswep']
Total sources: 4


## Aggregate Metrics by Model and Source

Group metrics by model type and meteorological source. Calculate mean and median values.

In [4]:
def aggregate_metrics(model_data_dict: dict) -> dict:
    """Aggregate metrics by meteo source.

    Args:
        model_data_dict: Dictionary {gauge_id: {meteo_source: metrics}}.

    Returns:
        Dictionary {meteo_source: {metric_name: list_of_values}}.
    """
    aggregated = {}

    for gauge_id, sources_dict in model_data_dict.items():
        for meteo_source, metrics in sources_dict.items():
            if meteo_source not in aggregated:
                aggregated[meteo_source] = {}

            for metric_name, metric_value in metrics.items():
                if metric_name not in aggregated[meteo_source]:
                    aggregated[meteo_source][metric_name] = []

                # Only add numeric values
                if isinstance(metric_value, (int, float)):
                    aggregated[meteo_source][metric_name].append(metric_value)

    return aggregated


# Aggregate metrics for each model
aggregated_data = {}
for model_name, model_dict in model_data.items():
    aggregated_data[model_name] = aggregate_metrics(model_dict)

print("Aggregation complete.")
print("\nMetrics available for each source:")
for model_name in aggregated_data:
    sample_source = next(iter(aggregated_data[model_name].keys()), None)
    if sample_source:
        metrics = aggregated_data[model_name][sample_source]
        print(f"  {model_name}: {list(metrics.keys())}")

Aggregation complete.

Metrics available for each source:
  GR4J: ['NSE', 'KGE', 'PBIAS', 'RMSE', 'MAE', 'logNSE', 'R2', 'PFE']


## Create Summary Statistics Table

Build a summary table with mean and median values for all metrics.

In [5]:
def build_summary_table(aggregated_data: dict) -> pd.DataFrame:
    """Build summary statistics table.

    Args:
        aggregated_data: Dictionary {model: {source: {metric: [values]}}}.

    Returns:
        DataFrame with rows as Model-Source combinations and columns as metrics.
    """
    rows = []

    for model_name, sources_dict in aggregated_data.items():
        for meteo_source, metrics_dict in sources_dict.items():
            row = {"Model": model_name, "Source": meteo_source}

            for metric_name, values in metrics_dict.items():
                if values:
                    values_array = np.array(values)
                    row[f"{metric_name}_median"] = np.nanmedian(values_array)
                    row[f"{metric_name}_mean"] = np.nanmean(values_array)

            rows.append(row)

    return pd.DataFrame(rows)


# Build the summary table
summary_df = build_summary_table(aggregated_data)
print(f"Summary table shape: {summary_df.shape}")
print(f"Columns: {list(summary_df.columns)}")

Summary table shape: (4, 18)
Columns: ['Model', 'Source', 'NSE_median', 'NSE_mean', 'KGE_median', 'KGE_mean', 'PBIAS_median', 'PBIAS_mean', 'RMSE_median', 'RMSE_mean', 'MAE_median', 'MAE_mean', 'logNSE_median', 'logNSE_mean', 'R2_median', 'R2_mean', 'PFE_median', 'PFE_mean']


## Display Comparison Results

Show the summary statistics table with performance metrics for all model and meteo source combinations.

In [ ]:
# Display full summary table
print("=" * 120)
print("SUMMARY STATISTICS: Model Performance by Meteorological Source")
print("=" * 120)

if len(summary_df) > 0:
    print(summary_df.to_string(index=False))
    print("=" * 120)

    # Create a pivot view for better comparison
    print("\n\nKEY FINDINGS:")
    print("-" * 120)

    # Get unique metrics
    all_columns = summary_df.columns.tolist()
    metric_names = set()
    for col in all_columns:
        if "_median" in col:
            metric_names.add(col.replace("_median", ""))

    print(f"Total model-source combinations: {len(summary_df)}")
    print(f"Unique metrics tracked: {len(metric_names)}")
    print(f"Metrics: {sorted(metric_names)}")
    print(f"\nModels: {summary_df['Model'].unique().tolist()}")
    print(f"Data sources: {summary_df['Source'].unique().tolist()}")
else:
    print("\n⚠️  No data loaded. Please ensure metrics files exist in:")
    for model_name, base_path in BASE_PATHS.items():
        print(f"   - {model_name}: {base_path}")
    print(
        "\nExpected file structure: {model_dir}/{gauge_id}/{gauge_id}_{source}_metrics.json"
    )
print("=" * 120)

SUMMARY STATISTICS: Model Performance by Meteorological Source
Model Source  NSE_median  NSE_mean  KGE_median  KGE_mean  PBIAS_median  PBIAS_mean  RMSE_median  RMSE_mean  MAE_median  MAE_mean  logNSE_median  logNSE_mean  R2_median  R2_mean  PFE_median   PFE_mean
 GR4J  mswep    0.431638 -2.577517    0.571123  0.219075      1.850627    6.968146     0.579088   0.789950    0.309604  0.427966       0.400735    -0.280626   0.591546 0.558832  -15.393852  -0.072156
 GR4J   gpcp    0.166081 -1.850846    0.193970  0.010754      3.387080   18.062986     0.642770   0.962069    0.416976  0.577994       0.037571    -0.495839   0.381519 0.387355  -43.299618 -28.161330
 GR4J    e5l    0.514289 -2.671481    0.623488  0.227880      1.160937    6.101205     0.550928   0.732269    0.279181  0.376203       0.539744    -0.288703   0.670012 0.613873   -9.752190   6.409423
 GR4J     e5    0.449891 -3.133478    0.592185  0.194092      1.029173    6.989282     0.563082   0.759800    0.288540  0.403599       0.